In [ ]:
from copy import deepcopy

# Data handling and plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Landlab and terrainbento
from landlab import RasterModelGrid, imshow_grid
from landlab.components import NormalFault
from landlab.values import constant, plane, sine, random
from terrainbento import Basic, BasicRt, Clock
from terrainbento.boundary_handlers import NotCoreNodeBaselevelHandler

# Own local functions
from output_writers import output_writer_1, print_model_local_time

In [ ]:
np.random.seed(103406)
fields = [
         'cumulative_elevation_change',
         'drainage_area',
         'flow__data_structure_delta',
         'flow__link_to_receiver_node',
         'flow__receiver_node',
         'flow__sink_flag',
         'flow__upstream_node_order',
         'initial_topographic__elevation',
         'rainfall__flux',
         'surface_water__discharge',
         'topographic__elevation',
         'topographic__steepest_slope',
         'water__unit_flux_in'
         ]

In [ ]:
# Set up the initial surface and boundary conditions
rmg = RasterModelGrid(shape=(800, 800),
                      xy_spacing=5,
                      xy_axis_units="m")

data = pd.read_csv('times_rates_rapid_fix.txt', sep='\t')

# z = constant(rmg, 'topographic__elevation', value=267.5)
# z = sine(rmg, 'topographic__elevation',
#          amplitude=0.02,
#          wavelength=500,
#          a=1,
#          b=0,
#          point=(-250, 0))

z = random(rmg, 'topographic__elevation', 
           distribution='normal')
z *= 2
rmg['node']['topographic__elevation'] += 267.5
p = 1 - rmg.y_of_node / rmg.y_of_node.max()
p *= 2
rmg['node']['topographic__elevation'] += p

In [ ]:
row_y_coords = np.unique(rmg.node_y)

fault_trace = {'x1': rmg.node_x[0],
               'x2': rmg.node_x[-1],
               'y1': row_y_coords[719],
               'y2': row_y_coords[719]}

# NormalFault simulating base level drop for the topmost nodes.
nf = NormalFault(grid=rmg,
                 faulted_surface='topographic__elevation',
                 fault_throw_rate_through_time={'rate': data['rate'],
                                                'time': data['timestep']},
                 fault_trace=fault_trace,
                 include_boundaries=True)



In [ ]:
# Close edge nodes, only leave top edge nodes open (core).
rmg.set_status_at_node_on_edges(right=rmg.BC_NODE_IS_CLOSED,
                                left=rmg.BC_NODE_IS_CLOSED,
                                top=rmg.BC_NODE_IS_CORE,
                                bottom=rmg.BC_NODE_IS_CLOSED,
                                )

In [ ]:
# Model setup
clock = Clock(0, 10000, 2940000)
model = Basic(clock,
              grid=rmg,
              boundary_handlers={'NormalFault': nf,
                                 # 'NotCoreNodeBaselevelHandler': bh
                                 },
              water_erodibility=0.00001,
              regolith_transport_parameter=0.01,
              output_interval=20000,
              fields=fields,
              output_prefix=output_prefix,
              output_writers={'function': [print_model_local_time]},
              n_sp=1,
              flow_director='FlowDirectorD8'
              )
model.run()

In [ ]:
# Plots and data extraction from the model.
profile_dist = rmg.y_of_node[rmg.x_of_node==10]
initial_profile = rmg['node']['initial_topographic__elevation'][rmg.x_of_node==10]
profile = rmg['node']['topographic__elevation'][rmg.x_of_node==10]
cross_profile = rmg['node']['topographic__elevation'][rmg.y_of_node==1500]
cross_profile_dist = rmg.x_of_node[rmg.y_of_node==1500]

plt.figure('Profil')
plt.title("Terrace evolution model profiles")
plt.plot(profile_dist, profile, label="Final")
plt.plot(profile_dist, initial_profile, label="Initial profile")
plt.xlabel('Distance from valley top [m]')
plt.ylabel('Elevation [m]')
plt.legend()
plt.show()

plt.figure('Topographic elevation')
imshow_grid(model.grid, 'topographic__elevation', cmap='terrain')

plt.figure('Initial topographic elevation')
imshow_grid(model.grid, 'initial_topographic__elevation', cmap='terrain')

output_writer_1(model)

In [ ]:
# Save stack of all netcdfs for Paraview to use.
model.save_to_xarray_dataset(filename=output_prefix+"_timed.nc",
                              time_unit="years",
                              reference_time="model start",
                              space_unit="meters")